In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Elective_Groups/Capstone/Step_2_transactions_dataset_outlier_removal.parquet')
df.info

<bound method DataFrame.info of          date_order product_id client_id    sales_net  quantity order_channel  \
0        2017-09-25    2376333    188502   155.443207         3        online   
1        2017-09-25    2536756   1977896   365.766235        21        online   
2        2017-09-25     203377   2086861  1093.374023         3      by phone   
3        2017-09-25     745693    482822     1.449000         5      by phone   
4        2017-09-25     683924    224223    12.033600         3  at the store   
...             ...        ...       ...          ...       ...           ...   
26061256 2019-07-19    1331967    181771 -3722.991699         3      by phone   
26061257 2019-07-19    1366829    294350    -0.441600         3      by phone   
26061258 2018-12-14     908253    880496    27.627600      1001      by phone   
26061259 2019-07-26    1168247   2063314  -204.609833       101      by phone   
26061260 2018-05-23    2319965   1287654    70.497299        17      by phone

In [ ]:
df.head()

,date_order,product_id,client_id,sales_net,quantity,order_channel,branch_id,month
0,2017-09-25,2376333,188502,155.443207,3,online,2732,2017-09
1,2017-09-25,2536756,1977896,365.766235,21,online,6184,2017-09
2,2017-09-25,203377,2086861,1093.374023,3,by phone,7051,2017-09
3,2017-09-25,745693,482822,1.449000,5,by phone,2771,2017-09
4,2017-09-25,683924,224223,12.033600,3,at the store,9463,2017-09


In [ ]:
df_customer = pd.DataFrame()

In [ ]:
model_customer = df[['client_id','date_order']].groupby('client_id').count().reset_index()
model_customer = model_customer.loc[model_customer.date_order > 0]
df_customer['client_id'] = model_customer['client_id']
df_customer.head()

,client_id
0,1000078
1,1000352
6,1000798
7,1001104
9,1001363


1.1 Recency

In [ ]:
max_date = df[['client_id','date_order']].groupby('client_id').max().reset_index()
df_customer = df_customer.merge(max_date,on='client_id',how='left')
df_customer.head()

,client_id,date_order
0,1000078,2019-03-22
1,1000352,2019-09-06
2,1000798,2019-09-18
3,1001104,2018-06-06
4,1001363,2019-09-19


In [ ]:
import datetime
testeddate = '2019/09/22'
NOW = datetime.datetime.strptime(testeddate,'%Y/%m/%d')

In [ ]:
df_customer["Recency"] = df_customer["date_order"].apply(lambda x: (NOW - x).days)
df_customer

,client_id,date_order,Recency
0,1000078,2019-03-22,184
1,1000352,2019-09-06,16
2,1000798,2019-09-18,4
3,1001104,2018-06-06,473
4,1001363,2019-09-19,3
...,...,...,...
148720,144085,2019-08-29,24
148721,1511299,2019-07-22,62
148722,450600,2019-08-06,47
148723,742185,2019-06-26,88


1.2 Frequency

In [ ]:
#amount_track = df.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
amount_track = df[['client_id','quantity']].groupby('client_id').sum().reset_index()
amount_track = amount_track.rename(columns={'quantity':'Total_Quantity'})
df_customer = df_customer.merge(amount_track,on='client_id',how='left')
df_customer

,client_id,date_order,Recency,Total_Quantity
0,1000078,2019-03-22,184,13361
1,1000352,2019-09-06,16,24795
2,1000798,2019-09-18,4,78138
3,1001104,2018-06-06,473,20112
4,1001363,2019-09-19,3,42807
...,...,...,...,...
148720,144085,2019-08-29,24,69
148721,1511299,2019-07-22,62,132
148722,450600,2019-08-06,47,18
148723,742185,2019-06-26,88,44


In [ ]:
#amount_track = df.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
refund_amount_track = df.loc[df.sales_net < 0][['client_id','quantity']].groupby('client_id').sum().reset_index()
refund_amount_track = refund_amount_track.rename(columns={'quantity':'R_Total_Quantity'})
df_customer = df_customer.merge(refund_amount_track,on='client_id',how='left')
df_customer

,client_id,date_order,Recency,Total_Quantity,R_Total_Quantity
0,1000078,2019-03-22,184,13361,3235
1,1000352,2019-09-06,16,24795,450
2,1000798,2019-09-18,4,78138,430
3,1001104,2018-06-06,473,20112,1569
4,1001363,2019-09-19,3,42807,540
...,...,...,...,...,...
148720,144085,2019-08-29,24,69,21
148721,1511299,2019-07-22,62,132,66
148722,450600,2019-08-06,47,18,3
148723,742185,2019-06-26,88,44,22


In [ ]:
df_customer["Frequency"] = df_customer["Total_Quantity"]-df_customer["R_Total_Quantity"]
df_customer.head()

,client_id,date_order,Recency,Total_Quantity,R_Total_Quantity,Frequency
0,1000078,2019-03-22,184,13361,3235,10126
1,1000352,2019-09-06,16,24795,450,24345
2,1000798,2019-09-18,4,78138,430,77708
3,1001104,2018-06-06,473,20112,1569,18543
4,1001363,2019-09-19,3,42807,540,42267


1.3 Monetary Value

In [ ]:
#amount_track = df.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
sales_total = df[['client_id','sales_net']].groupby('client_id').sum().reset_index()
sales_total = sales_total.rename(columns={'sales_net':'Total_Sales'})
df_customer = df_customer.merge(sales_total,on='client_id',how='left')
df_customer

,client_id,date_order,Recency,Total_Quantity,R_Total_Quantity,Frequency,Total_Sales
0,1000078,2019-03-22,184,13361,3235,10126,30274.775391
1,1000352,2019-09-06,16,24795,450,24345,78321.203125
2,1000798,2019-09-18,4,78138,430,77708,204420.250000
3,1001104,2018-06-06,473,20112,1569,18543,28551.833984
4,1001363,2019-09-19,3,42807,540,42267,44383.253906
...,...,...,...,...,...,...,...
148720,144085,2019-08-29,24,69,21,48,235.405930
148721,1511299,2019-07-22,62,132,66,66,0.000000
148722,450600,2019-08-06,47,18,3,15,0.000000
148723,742185,2019-06-26,88,44,22,22,0.000000


In [ ]:
#amount_track = df.groupby('client_id').agg({'quantity':['sum','mean'],'sales_net': ['count','sum','mean']}).reset_index()
refund_sales_total = df.loc[df.sales_net < 0][['client_id','sales_net']].groupby('client_id').sum().reset_index()
refund_sales_total = refund_sales_total.rename(columns={'sales_net':'R_Total_Sales'})
df_customer = df_customer.merge(refund_sales_total,on='client_id',how='left')
df_customer

,client_id,date_order,Recency,Total_Quantity,R_Total_Quantity,Frequency,Total_Sales,R_Total_Sales
0,1000078,2019-03-22,184,13361,3235,10126,30274.775391,-9034.760742
1,1000352,2019-09-06,16,24795,450,24345,78321.203125,-1809.559814
2,1000798,2019-09-18,4,78138,430,77708,204420.250000,-11392.935547
3,1001104,2018-06-06,473,20112,1569,18543,28551.833984,-2974.807129
4,1001363,2019-09-19,3,42807,540,42267,44383.253906,-1124.865601
...,...,...,...,...,...,...,...,...
148720,144085,2019-08-29,24,69,21,48,235.405930,-1232.886475
148721,1511299,2019-07-22,62,132,66,66,0.000000,-1549.754761
148722,450600,2019-08-06,47,18,3,15,0.000000,-2.290800
148723,742185,2019-06-26,88,44,22,22,0.000000,-129.070480


In [ ]:
df_customer["Monetary_Value"] = df_customer["Total_Sales"]+df_customer["R_Total_Sales"]

In [ ]:
R_2 = df_customer["Recency"].quantile(0.2)
R_4 = df_customer["Recency"].quantile(0.4)
R_6 = df_customer["Recency"].quantile(0.6)
R_8 = df_customer["Recency"].quantile(0.8)

def Recency_Quantitle(value):
  if value < R_2:
    return 5
  elif value < R_4:
    return 4
  elif value < R_6:
    return 3
  elif value < R_8:
    return 2
  else:
    return 1

In [ ]:
F_2 = df_customer["Frequency"].quantile(0.2)
F_4 = df_customer["Frequency"].quantile(0.4)
F_6 = df_customer["Frequency"].quantile(0.6)
F_8 = df_customer["Frequency"].quantile(0.8)

def Frequency_Quantitle(value):
  if value < F_2:
    return 5
  elif value < F_4:
    return 4
  elif value < F_6:
    return 3
  elif value < F_8:
    return 2
  else:
    return 1

In [ ]:
M_2 = df_customer["Monetary_Value"].quantile(0.2)
M_4 = df_customer["Monetary_Value"].quantile(0.4)
M_6 = df_customer["Monetary_Value"].quantile(0.6)
M_8 = df_customer["Monetary_Value"].quantile(0.8)

def Monetary_Value_Quantitle(value):
  if value < M_2:
    return 5
  elif value < M_4:
    return 4
  elif value < M_6:
    return 3
  elif value < M_8:
    return 2
  else:
    return 1

In [ ]:
df_customer["Recency_Quantitle"] = df_customer["Recency"].apply(lambda x: Recency_Quantitle(x))
df_customer["Frequency_Quantitle"] = df_customer["Frequency"].apply(lambda x: Frequency_Quantitle(x))
df_customer["Monetary_Value_Quantitle"] = df_customer["Monetary_Value"].apply(lambda x: Monetary_Value_Quantitle(x))

In [ ]:
df_customer['RFM'] = df_customer['Recency_Quantitle']*100 + df_customer['Frequency_Quantitle']*10 +df_customer['Monetary_Value_Quantitle']

In [ ]:
def Score_Quantitle(value):
    if value in [555, 554, 544, 545, 454, 455, 445]:
        return "Champion"
    elif value in [543, 444, 435, 355, 354, 345, 344, 335]:
        return "Loyal Customer"
    elif value in [553, 551, 552, 541, 542, 533, 532, 531, 452, 451, 442, 441, 431, 453, 433, 432, 423, 353, 352, 351, 342, 341, 333, 323]:
        return "Potential Loyalist"
    elif value in [512, 511, 422, 421, 412, 411, 311]:
        return "New Customer"
    elif value in [525, 524, 523, 522, 521, 515, 514, 513, 425, 424, 413, 414, 415, 315, 314, 313]:
        return "Promising"
    elif value in [535, 534, 443, 434, 343, 334, 325, 324]:
        return "Need Attention"
    elif value in [155, 154, 144, 214,215,115, 114, 113]:
        return "Cannot Lose Them"
    elif value in [331, 321, 312, 221, 213]:
        return "About To Sleep"
    elif value in [255, 254, 245, 244, 253, 252, 243, 242, 235, 234, 225, 224, 153, 152, 145, 143, 142, 135, 134, 133, 125, 124]:
        return "At Risk"
    elif value in [332, 322, 231, 241, 251, 233, 232, 223, 222, 132, 123, 122, 212, 211]:
        return "Hibernating"
    elif value in [111, 112, 121, 131, 141, 151]:
        return "Lost"



In [ ]:
df_customer["RFM_Segment"] = df_customer["RFM"].apply(lambda x: Score_Quantitle(x))

In [ ]:
df_customer

,client_id,date_order,Recency,Total_Quantity,R_Total_Quantity,Frequency,Total_Sales,R_Total_Sales,Monetary_Value,Recency_Quantitle,Frequency_Quantitle,Monetary_Value_Quantitle,RFM,RFM_Segment
0,1000078,2019-03-22,184,13361,3235,10126,30274.775391,-9034.760742,21240.015625,2,2,2,222,Hibernating
1,1000352,2019-09-06,16,24795,450,24345,78321.203125,-1809.559814,76511.640625,4,1,1,411,New Customer
2,1000798,2019-09-18,4,78138,430,77708,204420.250000,-11392.935547,193027.312500,5,1,1,511,New Customer
3,1001104,2018-06-06,473,20112,1569,18543,28551.833984,-2974.807129,25577.027344,1,1,2,112,Lost
4,1001363,2019-09-19,3,42807,540,42267,44383.253906,-1124.865601,43258.386719,5,1,1,511,New Customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148720,144085,2019-08-29,24,69,21,48,235.405930,-1232.886475,-997.480530,4,5,5,455,Champion
148721,1511299,2019-07-22,62,132,66,66,0.000000,-1549.754761,-1549.754761,3,5,5,355,Loyal Customer
148722,450600,2019-08-06,47,18,3,15,0.000000,-2.290800,-2.290800,3,5,5,355,Loyal Customer
148723,742185,2019-06-26,88,44,22,22,0.000000,-129.070480,-129.070480,2,5,5,255,At Risk


In [25]:
df_customer.to_csv('/content/drive/MyDrive/Elective_Groups/Capstone/Step_3_customer_dashboard.csv')